In [1]:
import pyrosetta
pyrosetta.init()

from multiprocessing import Pool
import logging
#logging.basicConfig(level=logging.INFO)

import itertools
import matplotlib
import pandas as pd
import seaborn

core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: 2018.51.post.dev+654.v2019.01dev60566.dbc838b6ae6 dbc838b6ae620b1293476b1bd4366ffc2facc5b5 ssh://git@github.com/RosettaCommons/main 2019-01-03T10:31:13
core.init: command: PyRosetta -ex1 -ex2aro -database /anaconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-1978162951 seed_offset=0 real_seed=-1978162951
core.init.random: RandomGenerator:init: Normal mode, seed=-1978162951 RG_type=mt19937


In [2]:
# import pose from pdb

pdb = '1stn-relaxed.pdb'
relaxed_pose = pyrosetta.pose_from_pdb(pdb)

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 636 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.913529 seconds.
core.import_pose.import_pose: File '1stn-relaxed.pdb' automatically determined to be of type PDB


In [14]:
# make backrub ensemble from WT

n_backrub = 50

backrub_protocol = pyrosetta.rosetta.protocols.backrub.BackrubProtocol()
backrub_pose_list = []

 for i in range(n_backrub):
    pose = relaxed_pose.clone()
    pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, "structure_num", i)
    backrub_protocol.apply(pose)
    backrub_pose_list.append(pose)

core.mm.MMBondAngleLibrary: MM bond angle sets added fully assigned: 603; wildcard: 0 and 1 virtual parameter.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.380838 seconds to load from binary


In [8]:
# read in mutations from Kellogg dataset

df = pd.read_csv('./kellogg.csv', comment = '#')
df['chain'], df['wt_res'], df['pdb_res_index'], df['mut_res'] = df['Mutations'].str.split(' ', 3).str
df.res_index = df.pdb_res_index.astype(int)
df = df.drop('Mutations', axis=1)
df.head()

/Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


,RecordID,PDBFileID,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,chain,wt_res,pdb_res_index,mut_res
0,1,107L,-0.53,0.435294117647,H,H,0.53,A,G,44,S
1,2,160L,-0.20,0.070796460177,H,H,0.2,A,A,120,M
2,3,161L,0.17,0.212389380531,H,H,-0.17,A,A,116,N
3,4,162L,-0.24,0.389380530973,H,H,0.24,A,A,122,Q
4,5,163L,-0.22,0.46017699115,T,O,0.22,A,A,123,Q


In [11]:
# pick mutations based on pdb

pdb_id = pdb[:4].upper()
pdb_ddgs = df[df.PDBFileID == pdb_id]
pdb_ddgs['res_index'] = pdb_ddgs.apply(lambda x: relaxed_pose.pdb_info().pdb2pose(x['chain'], int(x['pdb_res_index'])), axis=1)

print('PDB: {}'.format(pdb_id))
print('Max ddG = {}'.format(pdb_ddgs.DDG.max()))
print('Min ddG = {}'.format(pdb_ddgs.DDG.min()))

pdb_ddgs.head()


PDB: 1STN
Max ddG = 7.5
Min ddG = -1.25


/Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,RecordID,PDBFileID,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,chain,wt_res,pdb_res_index,mut_res,res_index
581,706,1STN,5.2,0.0,H,H,5.2,A,N,100,A,95
582,707,1STN,5.1,0.0,H,H,5.1,A,N,100,G,95
583,558,1STN,1.9,0.229508196721,H,H,1.9,A,E,101,A,96
584,559,1STN,3.1,0.229508196721,H,H,3.1,A,E,101,G,96
585,515,1STN,1.9,0.150442477876,H,H,1.9,A,A,102,F,97


In [12]:
# prepare mutation arguments from dataframe

mutations = list(zip(pdb_ddgs.res_index,
                     pdb_ddgs.mut_res))

In [ ]:
# run mutate and minimize, using multiprocessing

with pyrosetta.distributed.utility.log.LoggingContext(logging.getLogger("rosetta"), level=logging.WARN):
    with Pool() as p:
        work = [(structure_num, chain, wt_res, res_index, mut_res)
                for structure_num, (chain, wt_res, res_index, mut_res)
                in itertools.product(backrub_pose_list, mutations)]          
        logging.info("Now generating mutations and minimizing")
        all_structures = p.starmap(MUTATE_AND_MINIMIZE, work)
        
        
        

In [19]:
def mutate_minimize(input_pose, res_index, new_aa = None):
    '''Adapted from an example by Alex Ford and Brian Weitzner at Pre-RosettaCON 2018¶'''
    
    #work_pose = packed_pose.to_pose(input_pose)
    work_pose = input_pose
    
    #Mutate residue
    if new_aa != None:
        print('In IF STATEMENT')
        pyrosetta.toolbox.mutants.mutate_residue(work_pose, res_index, new_aa)
        pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_index", res_index)
    else:
        pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_aa", 'WT')
        pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_index", res_index)
    
    #Minimize inner shell and whole protein
    minimizer(work_pose, res_index)
    
    return work_pose

In [14]:
def minimizer(work_pose, res_index):
    
    #work_pose = pyrosetta.pose_from_pdb('/Users/mariajaime/Desktop/2018-2019/PyRosetta/ddG/pyrosetta_project/PyRosetta_Project/1stn-relaxed.pdb')
    #pmm = pyrosetta.PyMOLMover()
    #pmm.pymol_name('input')
    #pmm.apply(work_pose)

    # MovemapFactory
    mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
    mmf.all_bb(setting=True)  # Set to true if needed
    mmf.all_bondangles(setting=True)
    mmf.all_bondlengths(setting=True)
    mmf.all_chi(setting=True)
    mmf.set_cartesian(setting=True)
    #help(pyrosetta.rosetta.core.select.movemap.MoveMapFactory)

    # Create residue selectors

    index_selector = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector(str(res_index))
    close_residue_selector = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(index_selector, 10.0, True)
    not_close_residue_selector = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(close_residue_selector)

    # Create residue level task operations

    prevent_repacking_rlt = pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT()

    # Create task operations

    not_close_prevent_repack_t_op = pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset()
    #help(not_close_prevent_repack_t_op)
    not_close_prevent_repack_t_op.op(prevent_repacking_rlt)
    not_close_prevent_repack_t_op.selector(not_close_residue_selector)

    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.RestrictToRepacking()) #keep from designing
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())
    tf.push_back(not_close_prevent_repack_t_op)

    #Minimization

    sfxn = pyrosetta.create_score_function("ref2015_cart.wts")
    min_mover = pyrosetta.rosetta.protocols.minimization_packing.MinMover('lbfgs_armijo_nonmonotone')
    min_mover.min_type('lbfgs_armijo_nonmonotone')
    min_mover.tolerance(0.01)

    #help(min_mover)
    min_mover.movemap_factory(mmf) 
    min_mover.score_function(sfxn)
    #min_mover.apply(work_pose)

    # Packing

    packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover()
    packer.task_factory(tf)
    packer.apply(work_pose)

    #pmm.pymol_name('output')
    #pmm.apply(work_pose)
    
    return work_pose



In [27]:
sfxn = pyrosetta.create_score_function("ref2015_cart.wts")

relaxed_pose = pyrosetta.pose_from_pdb('/Users/mariajaime/Desktop/2018-2019/PyRosetta/ddG/pyrosetta_project/PyRosetta_Project/1stn-relaxed.pdb')
pmm = pyrosetta.PyMOLMover()
pmm.pymol_name('input')
pmm.apply(relaxed_pose)
print(sfxn(relaxed_pose))

relaxed_pose = mutate_minimize(relaxed_pose, 5, 'I')

pmm.pymol_name('output')
pmm.apply(relaxed_pose)
print(sfxn(relaxed_pose))


core.import_pose.import_pose: File '/Users/mariajaime/Desktop/2018-2019/PyRosetta/ddG/pyrosetta_project/PyRosetta_Project/1stn-relaxed.pdb' automatically determined to be of type PDB
core.scoring.CartesianBondedEnergy: Creating new peptide-bonded energy container (136)
-183.40175638917364
In IF STATEMENT
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
protocols.minimization_packing.PackRotamersMover: [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.pack_rotamers: built 89 rotamers at 13 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
-177.03881456467562


In [10]:
relaxed_pose.sequence()

'KLHKPPATLIKAIDGDTVKLMYKGQPMTFRLLLVDTPETKHPKKGVEKYGPEASAFTKKMVENAKKIEVEFDKGQRTDKYGRGLAYIYADGKMVNEALVRQGLAKVAYVYKPNNTHEQHLRKSEAQAKKEKLNIWS'